In [1]:
import pandas as pd
import numpy as np

from models.base import tune_logistic_regression_pipeline

import time

In [4]:
df = pd.read_csv('data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")

In [6]:
model_four_cols = ['id','click','hour','banner_pos','device_id','device_ip','device_model','device_conn_type','C14','C17','C20','C21']

In [ ]:
def create_user(df):
    """
    Adds a user column.
    """
    device_id_null = 'a99f214a'
    df['user'] = df.device_id.where(df.device_id != device_id_null,
                                    df.device_ip_model)
    return df

## Develop CountEncoder

In [27]:
from sklearn.base import BaseEstimator, TransformerMixin

class CountEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, col, count_col_name):
        """
        col: String - the name of the column to encode
        col_name: String - the name of a column to output the resulting count feature.
        """
        self.col = col
        self.count_col_name = count_col_name
        self.counts = None # a pd.Series
    
    def fit(self, X, y=None):
        self.counts = X.groupby(self.col).size().to_frame(self.count_col_name)
        return self
    
    def transform(self, X):
        X = pd.merge(X, counts, how='left', on=self.col)
        X = X.fillna({self.col: 0})
        # TODO: drop the raw feature or not
        return X

In [42]:
counts = df.groupby('device_ip').size().to_frame('device_ip_count')

In [43]:
pd.merge(df, counts, how='left', on='device_ip')

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C15,C16,C17,C18,C19,C20,C21,device_ip_model,user,device_ip_count
0,11066293153052526398,0,2014-10-23 02:00:00,1005,0,5b08c53b,7687a86e,3e814130,ecad2386,7801e8d9,...,300,250,2523,0,35,-1,221,4665f0a5_36d749e5,4665f0a5_36d749e5,1
1,8222349973801650597,1,2014-10-22 10:00:00,1005,0,5b08c53b,7687a86e,3e814130,ecad2386,7801e8d9,...,300,250,2295,2,35,100077,23,80df79b5_76dc4769,80df79b5_76dc4769,1
2,9513665962503843628,0,2014-10-21 17:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,320,50,2502,0,35,-1,221,693bff3e_4ea23a13,693bff3e_4ea23a13,2
3,7946799301127038469,0,2014-10-24 08:00:00,1005,0,1a5c1d83,5dddf09e,f028772b,ecad2386,7801e8d9,...,320,50,1973,3,39,-1,23,2773b1c8_d787e91b,2773b1c8_d787e91b,1
4,13112131110372814102,0,2014-10-22 19:00:00,1005,1,b7e9786d,b12b9f85,f028772b,ecad2386,7801e8d9,...,320,50,2526,0,35,100079,221,b200f41e_8a4875bd,b200f41e_8a4875bd,1
5,11893655186990579651,0,2014-10-29 22:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,0acbeaa3,45a51db4,...,320,50,2716,3,47,-1,23,2515535d_1f0bc64f,2515535d_1f0bc64f,1
6,2759258985299569696,0,2014-10-21 05:00:00,1005,0,7294ea0f,863fa89d,3e814130,ecad2386,7801e8d9,...,320,50,2333,0,39,-1,157,2da33ceb_fce66524,2da33ceb_fce66524,1
7,11000255552665064170,0,2014-10-23 15:00:00,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,216,36,2333,0,39,-1,157,815ad2c9_e981565c,815ad2c9_e981565c,1
8,2744263730495341601,1,2014-10-28 14:00:00,1002,0,75f43c5a,c4e18dd6,50e219e0,ecad2386,7801e8d9,...,320,50,2684,2,291,100084,52,6145ac58_13367cc6,e6af7320,1
9,16172535986144881057,0,2014-10-28 03:00:00,1010,1,85f751fd,c4e18dd6,50e219e0,6fef5db2,7801e8d9,...,320,50,2649,0,39,-1,204,b7a4f838_36b67a2a,02a76ae5,1


In [ ]:
df['device_ip_count'] = 

## Define user
user := ip & model if device_id is null (== a99f214a), else device_id.

In [7]:
user_cols = ['device_ip', 'device_model', 'device_id']

In [10]:
device_id_null = 'a99f214a'
(df.device_id == device_id_null).sum()

817

In [13]:
df['device_ip_model'] = df.device_ip.str.cat(df.device_model, sep='_')

In [15]:
df[user_cols + ['device_ip_model']].head()

,device_ip,device_model,device_id,device_ip_model
0,4665f0a5,36d749e5,a99f214a,4665f0a5_36d749e5
1,80df79b5,76dc4769,a99f214a,80df79b5_76dc4769
2,693bff3e,4ea23a13,a99f214a,693bff3e_4ea23a13
3,2773b1c8,d787e91b,a99f214a,2773b1c8_d787e91b
4,b200f41e,8a4875bd,a99f214a,b200f41e_8a4875bd


In [22]:
df['user'] = df.device_id.where(df.device_id != device_id_null, df.device_ip_model)

In [23]:
df[user_cols + ['device_ip_model', 'user']].head()

,device_ip,device_model,device_id,device_ip_model,user
0,4665f0a5,36d749e5,a99f214a,4665f0a5_36d749e5,4665f0a5_36d749e5
1,80df79b5,76dc4769,a99f214a,80df79b5_76dc4769,80df79b5_76dc4769
2,693bff3e,4ea23a13,a99f214a,693bff3e_4ea23a13,693bff3e_4ea23a13
3,2773b1c8,d787e91b,a99f214a,2773b1c8_d787e91b,2773b1c8_d787e91b
4,b200f41e,8a4875bd,a99f214a,b200f41e_8a4875bd,b200f41e_8a4875bd


In [26]:
df[df.device_id != device_id_null][user_cols + ['device_ip_model', 'user']].head()

,device_ip,device_model,device_id,device_ip_model,user
8,6145ac58,13367cc6,e6af7320,6145ac58_13367cc6,e6af7320
9,b7a4f838,36b67a2a,02a76ae5,b7a4f838_36b67a2a,02a76ae5
26,095745ae,43e7b962,306134d1,095745ae_43e7b962,306134d1
28,8bb32de9,9efa421a,6f1800fd,8bb32de9_9efa421a,6f1800fd
33,41d0f002,bf83adaa,56d28fb9,41d0f002_bf83adaa,56d28fb9
